In [ ]:
# https://docs.pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html

In [ ]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

# Tensor

In [ ]:
# pip install --upgrade --force-reinstall torch numpy 

In [ ]:
import torch
import numpy as np

print('torch版本:', torch.__version__); 
print('torch安装路径:', torch.__file__); 
print('numpy版本:', np.__version__); 
print('numpy安装路径:', np.__file__); 

In [ ]:
# 从数据
data = [[1, 2],[3, 4]]
x_data = torch.tensor(data)
print(x_data)
# 从NumPy数组
# np_array = np.array(data)
# x_np = torch.from_numpy(np_array)
# print(x_np)
# 从另一个张量
x_ones = torch.ones_like(x_data) 
print(f"Ones Tensor: \n {x_ones} \n")
x_rand = torch.rand_like(x_data, dtype=torch.float) 
print(f"Random Tensor: \n {x_rand} \n")

shape = (2,3,)
rand_tensor = torch.rand(shape)
ones_tensor = torch.ones(shape)
zeros_tensor = torch.zeros(shape)
print(f"Random Tensor: \n {rand_tensor} \n")
print(f"Ones Tensor: \n {ones_tensor} \n")
print(f"Zeros Tensor: \n {zeros_tensor}")

In [ ]:
# 一些属性
tensor = torch.rand(3,4)
print(f"Shape of tensor: {tensor.shape}")
print(f"Datatype of tensor: {tensor.dtype}")
print(f"Device tensor is stored on: {tensor.device}")

if torch.accelerator.is_available():
    tensor = tensor.to(torch.accelerator.current_accelerator())
print(f"Device tensor is stored on: {tensor.device}")

In [ ]:
tensor = torch.ones(4, 4)
print(f"First row: {tensor[0]}")
print(f"First column: {tensor[:, 0]}")
print(f"Last column: {tensor[..., -1]}")
tensor[:,1] = 0
print(tensor)

# 连接张量
t1 = torch.cat([tensor, tensor, tensor], dim=1)
print(t1)
# 算术运算
y1 = tensor @ tensor.T  # matrix multiplication @
# y2 = tensor.matmul(tensor.T)
# y3 = torch.rand_like(y1)
# torch.matmul(tensor, tensor.T, out=y3)
print(f"y1 == {y1}")
z1 = tensor * tensor    # element-wise mul *
# z2 = tensor.mul(tensor)
# z3 = torch.rand_like(tensor)
# torch.mul(tensor, tensor, out=z3)
print(f"z1 == {z1}")
# 单元素张量转换为Python数值 item()
agg = tensor.sum()
agg_item = agg.item()
print(f"agg_item: {agg_item, type(agg_item)}")
# 就地操作,使用 _后缀 # 可节省内存,但可能出问题
print(f"add_before: {tensor} ")
tensor.add_(5)
print(f"add_after: {tensor} ")

# Dataset & Datasetloader

In [ ]:
# pip install --upgrade --force-reinstall matplotlib torchvision pandas

In [ ]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt

# 加载已有数据集
training_data = datasets.FashionMNIST(
    root="data",# 存储路径
    train=True,# 指定训练或测试数据集
    download=True,# 如果数据不可用，请从internet下载
    transform=ToTensor()# 内置转化器 
)
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)
# 迭代和可视化数据集
labels_map = {
    0: "T-Shirt",
    1: "Trouser",
    2: "Pullover",
    3: "Dress",
    4: "Coat",
    5: "Sandal",
    6: "Shirt",
    7: "Sneaker",
    8: "Bag",
    9: "Ankle Boot",
}
figure = plt.figure(figsize=(8, 8))
cols, rows = 3, 3
for i in range(1, cols * rows + 1):
    sample_idx = torch.randint(len(training_data), size=(1,)).item()
    img, label = training_data[sample_idx]
    figure.add_subplot(rows, cols, i)
    plt.title(labels_map[label])
    plt.axis("off")
    plt.imshow(img.squeeze(), cmap="gray")
plt.show()

In [ ]:
# 自定义数据集类必须实现三个函数:__init__,__len__,和__getitem__
import os
import pandas as pd
from torchvision.io import decode_image

class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = decode_image(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [ ]:
# 使用DataLoaders
from torch.utils.data import DataLoader

train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)# shuffle=True 混洗数据
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)

train_features, train_labels = next(iter(train_dataloader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")
img = train_features[0].squeeze()   # .squeeze()压缩一个维度
label = train_labels[0]
plt.axis("off")
plt.imshow(img, cmap="gray")
plt.show()
print(f"Label: {label}")

# 分批读取
epochs = 10 # 10轮 
for i in range(epochs):
    for batch_idx,(batch_x,batch_y) in enumerate(train_dataloader):
        print(f"epoch[{i}] batch[{batch_idx}]")

In [ ]:
# 内置转换器: transform修改格式, target_transform修改标签 # 钩子函数
import torch
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

ds = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    # ToTensor(): PIL图像 or NumPyndarray -> FloatTensor[0,1]
    transform=ToTensor(),
    # (28,28,1) -> (1,28,28)
    # 图像(高,宽,通道) -> tensor(通道,高,宽) 
    target_transform=Lambda(lambda y: torch.zeros(10, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
    # 4 -> tensor[0,0,0,1,0,0]
    # 标签整数 -> one-hot张量
)

# Neural Network


In [ ]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using device: {device}")

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten() # x=(batch=2,28*28)
        self.linear_relu_stack = nn.Sequential( # 打包计算操作
            nn.Linear(28*28, 512),  # Linear层 x*W+b: 参数W=(28*28,512),b=(512) -> (2,512)   # 线性
            nn.ReLU(),              # 激活函数ReLU(), 负数全变0                     # 非线性
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )
        self.softmax = nn.Softmax(dim=1)    # Softmax层: 分别求X_i/sum(e^X_j) -> 概率[0,1]

    def forward(self, x):
        x = self.flatten(x)# 展平(512,1,28,28) -> x=(512,1*28*28)
        logits = self.linear_relu_stack(x)# -> (512, 10)
        prob = self.softmax(logits)
        return prob

model = NeuralNetwork().to(device)# 实例化model

# 实例测试
X = torch.rand(2, 28, 28, device=device)
# print(X.shape,X.dtype)
prob = model(X)
y_pred = prob.argmax(1) # axis=1 # 轴1上最大值的下标
print(f"y_pred == {y_pred}\n")
# 可视化模型参数
print(f"Model structure: {model}\n")
for name, param in model.named_parameters():# 层名,可学习参数
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} | requires_grad == {param.requires_grad}")

# Autograd

In [ ]:
import torch

x = torch.ones(5)  # input tensor
y = torch.zeros(3)  # expected output
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = x @ w+b
# 正向传播
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)
print(f"loss.grad_fn == {loss.grad_fn}")# loss的前一个算子
# 反向传播
loss.backward()
print(f"w == \n{w}")
print(f"w.grad == \n{w.grad}")
print(f"b.grad == {b.grad}")

print(f"\nBEFORE: z.requires_grad == {z.requires_grad}")
z.detach_() # detach()禁用梯度跟踪
print(f"AFTER: z.requires_grad == {z.requires_grad}")

# Optimizing Model Parameters

In [ ]:
# 前置代码
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

In [ ]:
# 超参数Hyperparameters:
learning_rate = 1e-3
batch_size = 64
epochs = 5
# 常用损失函数: 
# nn.MSELoss(均方误差) 用于回归任务
# nn.NLLLoss(负对数似然) 用于分类
loss_fn = nn.CrossEntropyLoss()# 交叉商损失函数, 联合nn.LogSoftmax和nn.NLLLoss
# 优化器，用于控制学习率lr
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)# SGD随机梯度下降

In [ ]:
# 循环loop: train_loop, test_loop
def train_loop(dataloader, model, loss_fn, optimizer):
    model.train()   # 模型设为train模式(此处非必要)
    print("training...")
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        pred = model(X) # pred = (batch_size= 64, y_pred= 10)
        loss = loss_fn(pred, y)
        # 反向传播
        loss.backward()
        optimizer.step()        # 根据lr优化, 如 w -= lr * w_grad
        optimizer.zero_grad()   # 归零param_grad, 防止grad自动叠加

        if batch % 100 == 0:# 每100*batch打印一次
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"Batch[{batch}]:\tloss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    model.eval()# 模型设为evaluation模式(此处非必要)
    print("evaluating...")
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():           # test时, 前向传播不再记录grad, 节约显存
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

for t in range(epochs):
    print(f"Epoch {t+1}\t-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

# Save and Load the Model

In [ ]:
import torch
import torchvision.models as models

In [ ]:
# 保存模型和参数
model = models.vgg16(weights='IMAGENET1K_V1')   
torch.save(model, 'model-vgg16.pth')             


In [ ]:
# 加载模型
model = torch.load('model-vgg16.pth', weights_only=False)
model.eval()    # 将dropout层和batch规一层设置为评估模式，防止过拟合 
print(model)
for name,param in model.named_parameters():
    print(name,param.shape)